In [8]:
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import re
import jellyfish
pd.set_option('display.precision',15)
from pyjarowinkler import distance
import os
wdir = os.getcwd()
wdir = wdir + '/'
df_npa = pd.read_csv(wdir+'DupsData/NPA_ForFuzzy.csv', index_col=0)
df_master = pd.read_csv(wdir+'Master/Master.csv')


cols_practice_npa = ['Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address']
cols_practice_master = ['Practice Street Address 1', 'Practice Street Address 2']
df_master['check1'] = df_master[cols_practice_master].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['check1'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)


cols_practice_npa = ['Provider First Line Business Mailing Address','Provider Second Line Business Mailing Address']
cols_practice_master = ['Shipping Address Line 1', 'Shipping Address Line 2', 'Shipping Address Line 3']
df_master['check2'] = df_master[cols_practice_master].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['check2'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)

df_npa = df_npa.drop_duplicates(subset=['check1', 'Provider Business Practice Location Address Telephone Number'])

businesspractices_npa = df_npa['Provider Business Practice Location Address Telephone Number'].tolist()
businesspractices_npa = list(set(businesspractices_npa))

filter = df_master["check1"] != ""
df_master1 = df_master[filter]

df_master1 = df_master1.dropna(subset=['check1'])
df_master1 = df_master1.dropna(subset=['Practice Phone'])


index_jf = []
businesspracticephone = df_npa['Provider Business Practice Location Address Telephone Number'].tolist()
for index,row in df_npa.iterrows():
    if row['Provider Business Practice Location Address Telephone Number'] in businesspractices_npa:
        f1 = re.sub('[^a-zA-Z0-9]', '', row['check1'])
        checkvalue = df_master1.loc[df_master1['Practice Phone'] == row['Provider Business Practice Location Address Telephone Number'], 'check1'].values
        if len(checkvalue)>0:
            f2 = re.sub('[^a-zA-Z0-9]', '', checkvalue[0])
             #check3 = re.sub('[^a-zA-Z0-9]', '', row['fullname'])
            #check4 = re.sub('[^a-zA-Z0-9]', '', row1['fullname'])
            if jellyfish.jaro_winkler(f1,f2)>.90:
                index_jf.append(index)   
                f = open(wdir+'DupsData/fuzzyvalues.txt', 'a')
                f.write(str(index) + ',' + f1 + ',' + f2 + '\n')
                f.close 
                        
fuzzindex = []
fuzzdata = pd.read_csv(wdir+'DupsData/fuzzyvalues.txt', header = None)
fuzzdata.columns = ['a','b','c']

                        
                        
for index,row in fuzzdata.iterrows():
    if str(row.b[0:8]) == str(row.c[0:8]):
        if len(re.findall(r'\d+', row.b))>=2 & len(re.findall(r'\d+', row.c))>=2:
            if (re.findall(r'\d+', row.b)[1] == re.findall(r'\d+', row.c)[1]):
                fuzzindex.append(row.a)
                f = open(wdir+'DupsData/fuzzyvalues-11.txt', 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close 
        if len(re.findall(r'\d+', row.b))==1 & len(re.findall(r'\d+', row.c))==1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex.append(row.a)
                f = open(wdir+'DupsData/fuzzyvalues-11.txt', 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
                
        if len(re.findall(r'\d+', row.b))==1 & len(re.findall(r'\d+', row.c))>1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex.append(row.a)
                f = open(wdir+'DupsData/fuzzyvalues-11.txt', 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
                
        if len(re.findall(r'\d+', row.b))>1 & len(re.findall(r'\d+', row.c))==1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex.append(row.a)
                f = open(wdir+'DupsData/fuzzyvalues-11.txt', 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
print(len(fuzzindex))
                        

3590
